In [1]:
import re
import os
import string

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from math import log

import pandas as pd
import numpy as np

dictionary = {}

docIdMap = {}
kata = []
base = []
child = []
namaDokumen = ""

In [2]:
def proses_pertama(path):
    docId = 1
    """
            Membuat pengulangan sesuai jumlah data txt pada suatu folder
    """
    fileList = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    print(fileList)
    fileobj = open('frekuensi.txt', 'w')
    for eachFile in fileList:
        position = 1
        count = 0
        """
        Di variable bawah ini nanti akan dimasukkan id File dan nama File txtnya
        """
        docIdMap[docId] = eachFile
        """
        DI variable bawah ini kita akan mendapatkan kalimat dari tiap barisnya
        """
        lines = [line.rstrip('\n') for line in open(path + "/" + eachFile)]
#         print(lines)        
#         print("================buat pengulangan")
        for eachLine in lines:
            """
            case folding
            """
            eachLine = eachLine.lower()
            eachLine = re.sub(r"\d+", "", eachLine)
            eachLine = eachLine.translate(str.maketrans("","",string.punctuation))
            #hapus simbol
            eachLine = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',eachLine)
            eachLine = re.sub('rt @[^\s]+',' ', eachLine)
            eachLine = re.sub('@[^\s]+',' ',eachLine)
            eachLine = re.sub(r'[^\x00-\x7F]+',' ', eachLine)
            #Remove additional white spaces
            eachLine = re.sub('[\s]+', ' ', eachLine)
            eachLine = re.sub(r'#([^\s]+)', r'\1', eachLine)
            eachLine = eachLine.strip()
            """
            stop words
            """
            stop_words = set(stopwords.words('english')) 
  
            word_tokens = word_tokenize(eachLine) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
#             print("==========================================STEMMING")
            """
            Stemming
            """
            ps = PorterStemmer()
            for k in filtered_sentence:
                kata.append(ps.stem(k))
#             for word in filtered_sentence:
                word = ps.stem(k)
                """
                    memasukkan ke dict,
                    nantinya biar dihitung frekeunsi tiap corpus
                """
                if (word.lower() in dictionary):
                    postingList = dictionary[word.lower()]
                    if (docId in postingList):
                        postingList[docId].append(position)
                        position = position + 1
                    else:
                        postingList[docId] = [position]
                        position = position + 1
                else:
                    dictionary[word.lower()] = {docId: [position]}
                    position = position + 1
#             print("==========================================END")
                    
        docId = docId + 1
            
    length_dict = {key: len(value) for key, value in dictionary.items()}
    """
        PROSES MENGHITUNG FREKUENSI TIAP FILE CORPUS
        dimasukkan ke dalam array child
    """
    for key in dictionary:
#         print(key)
        child.append(key)
#         for s, value in dictionary[key].items():
        jumlah = 0
        for sw in docIdMap:
            pass
            
            for s, value in dictionary[key].items():
                if (str(s) in str(sw)):
#                     print("pada dok "+str(sw) +" "+str(len(dictionary[key][s])))
                    child.append(len(dictionary[key][s]))
                    jumlah += 1
                    break
            else:
#                 print("pada dok gagal "+str(sw) +" 0")
                child.append(0)
        child.append(jumlah)
        child.append(log(len(docIdMap) / (jumlah), 10))
        base.append(child)

    """
    Buat file semua term pada semua corpus !
    """
    for w in length_dict:
        # ini nanti dibuat menangkap jumlah kata di suatu file txt
        fileobj.write(w +"   |   "+str(length_dict[w]))
        fileobj.write("\n")
    fileobj.close()
    print("================================================")
    print(child)

In [3]:
def main():
    AlamatDir = "test"
    queryFile = "query"
    proses_pertama(AlamatDir)

In [4]:
if __name__ == '__main__':
    main()

['business.txt', 'entertainment.txt', 'politics.txt']
['worldcom', 8, 0, 0, 1, 0.47712125471966244, 'boss', 4, 0, 0, 1, 0.47712125471966244, 'left', 1, 0, 0, 1, 0.47712125471966244, 'book', 2, 0, 1, 2, 0.17609125905568124, 'alon', 1, 0, 0, 1, 0.47712125471966244, 'former', 5, 0, 2, 2, 0.17609125905568124, 'berni', 1, 0, 0, 1, 0.47712125471966244, 'ebber', 7, 0, 0, 1, 0.47712125471966244, 'accus', 1, 0, 1, 2, 0.17609125905568124, 'overse', 1, 0, 0, 1, 0.47712125471966244, 'bn', 7, 0, 1, 2, 0.17609125905568124, 'fraud', 3, 0, 0, 1, 0.47712125471966244, 'never', 1, 1, 0, 2, 0.17609125905568124, 'made', 2, 1, 2, 3, 0.0, 'account', 5, 0, 0, 1, 0.47712125471966244, 'decis', 3, 0, 0, 1, 0.47712125471966244, 'wit', 2, 0, 0, 1, 0.47712125471966244, 'told', 4, 2, 6, 3, 0.0, 'juror', 1, 0, 0, 1, 0.47712125471966244, 'david', 2, 0, 0, 1, 0.47712125471966244, 'myer', 5, 1, 0, 2, 0.17609125905568124, 'comment', 1, 0, 1, 2, 0.17609125905568124, 'question', 1, 0, 0, 1, 0.47712125471966244, 'defenc', 3

In [5]:
data = [child[i:i+6] for i in range(0, len(child),6)]
kategori = ['business','entertainment','politics','DF','IDF']
kolom = []
p = []
kolom.append('Term / kata')

for x in docIdMap:
    p.append(str(kategori[x-1]))
    kolom.append(str(kategori[x-1]))
kolom.append('DF')
kolom.append('IDF')
print(docIdMap)
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = kolom) 
export = df.sort_values(p, ascending=False)
export
# print dataframe. 
# print(pd.concat([pd.DataFrame({})] + [df], axis=1))

{1: 'business.txt', 2: 'entertainment.txt', 3: 'politics.txt'}


,Term / kata,business,entertainment,politics,DF,IDF
26,mr,12,0,26,2,0.176091
37,firm,11,0,0,1,0.477121
132,year,9,5,4,3,0.000000
121,virgin,9,0,0,1,0.477121
122,blue,9,0,0,1,0.477121
...,...,...,...,...,...,...
1193,hors,0,0,1,1,0.477121
1194,ride,0,0,1,1,0.477121
1195,accid,0,0,1,1,0.477121
1196,remark,0,0,1,1,0.477121


In [13]:
df

,Term / kata,business,entertainment,politics,DF
0,worldcom,8,0,0,1
1,boss,4,0,0,1
2,left,1,0,0,1
3,book,2,0,1,2
4,alon,1,0,0,1
...,...,...,...,...,...
1193,hors,0,0,1,1
1194,ride,0,0,1,1
1195,accid,0,0,1,1
1196,remark,0,0,1,1


Jika kita ingin mencari term/kata

In [6]:
df[df['Term / kata'].str.contains("book")]

,Term / kata,business,entertainment,politics
3,book,2,15,12
4918,songbook,0,0,1
5029,bookmak,0,0,2
5270,booker,0,0,1


Jika ingin export ke csv file

In [7]:
export.to_csv (r'hasil-baru.csv', index = False, header=True)

In [9]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = kolom

for z in data:
    x.add_row(z)

# print(x)
# print(x.get_string(sortby="Term / kata", reversesort = False))

In [6]:
df

,Term / kata,business,entertainment,politics
0,worldcom,8,0,0
1,boss,4,0,0
2,left,1,0,0
3,book,2,0,1
4,alon,1,0,0
...,...,...,...,...
1193,hors,0,0,1
1194,ride,0,0,1
1195,accid,0,0,1
1196,remark,0,0,1


In [17]:
len(docIdMap)

3